In [5]:
import numpy as np
import tensorflow as tf
import strawberryfields as sf
from numba import njit

In [6]:
sf.__version__

'0.15.1'

In [26]:
cutoff_dim = 100
gamma1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)
phi1 = np.random.normal(scale=0.01)
zeta1 = np.random.normal(scale=0.01) + 1j*np.random.normal(scale=0.01)
Psi = np.zeros(cutoff_dim,dtype=np.complex128)
Psi[0] = 1  #vaccum

In [27]:
zeta1

(0.017346761383280025-0.000409972094980348j)

In [28]:
eng = sf.Engine('fock', backend_options={'cutoff_dim':cutoff_dim})

prog = sf.Program(1)
gamma1r = np.abs(gamma1)
gamma1phi = np.angle(gamma1)

with prog.context as mode:
    sf.ops.Ket(Psi)
    sf.ops.Dgate(gamma1r,phi=gamma1phi) | mode[0]
    sf.ops.Rgate(phi1) | mode[0]
    sf.ops.Sgate(zeta1) | mode[0]
state_out0 = eng.run(prog).state.ket()

In [39]:
import numpy as np
from numba import jit

@njit  # (nb.types.Tuple((nb.complex128, nb.complex128[:], nb.complex128[:,:]))(nb.complex128, nb.float64, nb.complex128))
def C_mu_Sigma(gamma: np.complex, phi: np.float, z: np.complex) -> tuple:
    """
    Utility function to construct:
    1. C constant
    2. Mu vector
    3. Sigma matrix
    Arguments:
        gamma (complex): displacement parameter
        phi (float): phase rotation parameter
        z (complex): squeezing parameter
        dtype (numpy type): unused for now
    Returns:
        C (complex), mu (complex array[2]), Sigma (complex array[2,2])
    """
#     z = convert_scalar(z)
#     phi = convert_scalar(phi)
#     gamma = convert_scalar(gamma)
    r = np.abs(z)
    delta = np.angle(z)
    exp2phidelta = np.exp(1j * (2 * phi + delta))
    eiphi = np.exp(1j * phi)
    tanhr = np.tanh(r)
    coshr = np.cosh(r)
    cgamma = np.conj(gamma)

    C = np.exp(-0.5 * np.abs(gamma) ** 2 - 0.5 * cgamma ** 2 * exp2phidelta * tanhr) / np.sqrt(coshr)
    mu = np.array([cgamma * exp2phidelta * tanhr + gamma, -cgamma * eiphi / coshr,])
    Sigma = np.array([[exp2phidelta * tanhr, -eiphi / coshr], [-eiphi / coshr, -np.exp(-1j * delta) * tanhr],])

    return C, mu, Sigma


@njit
def R_matrix(gamma: np.complex, phi: np.float, z: np.complex, cutoff: int, old_state: np.array) -> np.array:
    """
    Directly constructs the transformed state recursively and exactly.

    Arguments:
        gamma (complex): displacement parameter
        phi (float): phase rotation parameter
        z (complex): squeezing parameter
        old_state (complex array[D]): State to be transformed

    Returns:
        R (complex array[D,D]): the matrix whose 1st column is the transformed state
    """

    dtype = old_state.dtype
    C, mu, Sigma = C_mu_Sigma(gamma, phi, z)

    sqrt = np.sqrt(np.arange(cutoff, dtype=dtype))

    R = np.zeros((cutoff, cutoff), dtype=dtype)
    G0 = np.zeros(cutoff, dtype=dtype)

    # first row of Transformation matrix
    G0[0] = C
    for n in range(1, cutoff):
        G0[n] = mu[1] / sqrt[n] * G0[n - 1] - Sigma[1, 1] * sqrt[n - 1] / sqrt[n] * G0[n - 2]

    # first row of R matrix
    for n in range(cutoff):
        R[0, n] = np.dot(np.conj(G0[: cutoff - n]), old_state)
        old_state = old_state[1:] * sqrt[1 : cutoff - n]

    # rest of R matrix
    for m in range(1, cutoff):
        for n in range(cutoff - m):
            R[m, n] = (
                mu[0] / sqrt[m] * R[m - 1, n]
                - Sigma[0, 0] * sqrt[m - 1] / sqrt[m] * R[m - 2, n]
                - Sigma[0, 1] / sqrt[m] * R[m - 1, n + 1]
            )

    return R

In [40]:
state_out = R_matrix(gamma1,phi1, zeta1,cutoff_dim, Psi)[:,0]


In [38]:
np.sum((state_out0 - state_out)**2)

(4.0833356900980026e-08+7.294236810062155e-08j)

In [41]:
np.sum((state_out0 - state_out)**2)

(4.0830655682287484e-08+7.294254399797915e-08j)

In [42]:
state_out

array([ 9.99774672e-01-1.75020970e-07j,  1.74693124e-02+2.38862260e-04j,
       -1.20404258e-02+4.81275781e-04j, -3.68949754e-04+9.40440047e-06j,
        1.77271801e-04-1.41994668e-05j,  7.10224236e-06-4.59747326e-07j,
       -2.74613003e-06+3.30954483e-07j, -1.31876410e-07+1.37385142e-08j,
        4.35102886e-08-7.02070676e-09j,  2.40083332e-09-3.45592584e-10j,
       -6.97850032e-10+1.41498621e-10j, -4.31509797e-11+7.94710708e-12j,
        1.12741355e-11-2.76086662e-12j,  7.68437863e-13-1.72882720e-13j,
       -1.82959268e-13+5.26705449e-14j, -1.35869150e-14+3.62114556e-15j,
        2.97735071e-15-9.88257731e-16j,  2.38833644e-16-7.37837812e-17j,
       -4.85299066e-17+1.83057492e-17j, -4.17743281e-18+1.47209534e-18j])

In [43]:
state_out0

array([ 9.99779866e-01+3.65235195e-06j,  1.71707222e-02+1.25024977e-04j,
       -1.20535719e-02+2.92734155e-04j, -3.62757015e-04+6.00823465e-06j,
        1.77840134e-04-8.64659209e-06j,  6.99197459e-06-2.82661229e-07j,
       -2.76360614e-06+2.01819985e-07j, -1.30121552e-07+8.37168488e-09j,
        4.39710600e-08-4.28890922e-09j,  2.37655455e-09-2.09904022e-10j,
       -7.08957845e-10+8.66243385e-11j, -4.28955010e-11+4.82190650e-12j,
        1.15265421e-11-1.69437218e-12j,  7.67899646e-13-1.04917223e-13j,
       -1.88458191e-13+3.24162464e-14j, -1.36628167e-14+2.19980282e-15j,
        3.09341792e-15-6.10174962e-16j,  2.41935447e-16-4.48967632e-17j,
       -5.09199579e-17+1.13428080e-17j, -4.26749767e-18+8.97709427e-19j])